# Create summary plots for locations

This notebook is for creating some summmary plots for the various locations of interest.

The goal is to create barplots of thirteen indicators for each location, which we have summarized over eras and decades.

The barplots will show different types of summaries of the above indices across different time spans (main bars) as well as between-model variation (whiskers).

## Plotting

Run this cell first to set up the environment:

In [5]:
import warnings
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
# project
from config import *


# for runtime warning that is happening in DataFrame.plot() 
warnings.simplefilter(action='ignore', category=RuntimeWarning)
# for user warning with pyplot.tight_layout plus added axis
warnings.simplefilter(action='ignore', category=UserWarning)

### Decadal summary plots

This section will create barplots by decade for all locations and all available indices listed above. This is only done for the mean aggregation though, not the maximum.

Define a function to create a barplot from a dataframe of decadal summaries of the indices.

In [6]:
# not sure of a better way to change legend title size atm
plt.rcParams["legend.title_fontsize"] = 14

def make_decade_barplot(df, index, aggr_var, title, ylab):
    colors = ["#d2b48c", "#1f77b4", "#ff7f0e"]
     
    temp_df = (
        df[df["idx_var"] == index]
        .groupby(["scenario", "decade"])
        .agg({aggr_var: ["min", "mean", "max"]})
    )

    # drop column multiindex
    temp_df = temp_df.droplevel(0, axis=1).reset_index()
    avg_df = temp_df.pivot(index="decade", columns="scenario", values="mean")
    # error bar limits
    ll = (avg_df - temp_df.pivot(index="decade", columns="scenario", values="min")).to_numpy().T
    ul = (temp_df.pivot(index="decade", columns="scenario", values="max") - avg_df).to_numpy().T
    err = np.array([
        [ll[0], ul[0]], #hist limits 
        [ll[1], ul[1]], #rcp45 limits 
        [ll[2], ul[2]], #rcp85 limits
    ])
    fig, ax = plt.subplots(1, figsize=(14, 7))
    avg_df.plot(
        kind="bar", rot=0, yerr=err, ax=ax, capsize=5, legend=False, color=colors, width=1,
    )  # .legend(loc="upper left")

    for tick in ax.xaxis.get_major_ticks()[1::2]:
        tick.set_pad(15)
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.set_ylabel(ylab, size=14, rotation=0, labelpad=15)
    ax.set_xlabel("Decade", size=14)
    leg_ax = fig.add_axes([1.05, 0.2, 0.02, 0.7])
    handles = [
        mpatches.Patch(color=colors[0], label="Historical"),
        mpatches.Patch(color=colors[1], label="RCP 4.5"),
        mpatches.Patch(color=colors[2], label="RCP 8.5"),
    ]
    leg_ax.set_axis_off()
    # Add legend to bottom-right ax
    leg_ax.legend(handles=handles, loc="center", fontsize=14, title="Scenario")
    fig.suptitle(title, size=16)
    plt.tight_layout()

Set up some strings for help with plotting and saving:

In [3]:
tmp_fn = "barplot_{}_{}_{}.png"
title_template = "Mean {} by decade across all models for {}, AK"
plot_lu = {
    "rx1day": {"title": "Rx1day - max 1-day precip", "ylab": "mm",},
    "rx5day": {"title": "Rx5day - max 5-day precip", "ylab": "mm",},
    "r10mm": {"title": "R10mm - very wet days", "ylab": "Days",},
    "hsd": {"title": "Heavy snow days", "ylab": "cm",},
    "wsdi": {"title": "Warm spell duration index", "ylab": "Days",},
    "csdi": {"title": "Cold spell duration index", "ylab": "Days",},
    "hd": {"title": "Hot day", "ylab": "°C",},
    "cd": {"title": "Cold day", "ylab": "°C",},
    "su": {"title": "Summer days (max temp > 25 °C)", "ylab": "Days",},
    "dw": {"title": "Deep winter days (min temp < -30 °C)", "ylab": "Days",},
    "cdd": {"title": "Consecutive dry days (total precip > 1mm/day)", "ylab": "Days",},
    "cwd": {"title": "Consecutive wet days (total precip < 1mm/day)", "ylab": "Days",},
    "wndd": {"title": "Windy days (mean speed > 10 m/s)", "ylab": "Days",},
}

And iterate over locations and indices and plot:

In [7]:
for location in locations:
    df = pd.read_excel(idx_decade_summary_fp, location, engine="openpyxl") 
    for index in np.unique(df["idx_var"].values):
        title_str = title_template.format(plot_lu[index]["title"], location)
        ylab = plot_lu[index]["ylab"]
        
        # keeping this snippet just in case. Moving to means only for now
        # for aggr_var in ["min", "mean", "max"]:
        aggr_var = "mean"
        
        out = make_decade_barplot(df, index, aggr_var, title_str, ylab)
        if out == "skip":
            continue
        out_fp = decadal_summary_dir.joinpath(
            "barplots",
            index,
            tmp_fn.format(aggr_var, index, location)
        )
        out_fp.parent.mkdir(exist_ok=True, parents=True)
        plt.savefig(out_fp, bbox_inches='tight', facecolor="white")
        plt.close()
    print(f"{location} done")

Kaktovik done
Stevens Village done
Igiugik Village done
Levelock done
Eyak done
Ketchikan done
Aleutians done
Nanwalek done
Port Graham done
Qutekcak (Seward) done
Chenega Bay done
Tatitek done
Valdez done
Cordova done


Note - we do not print out the figures to the notebook here because there are many!